# <span style="color:deeppink">Linear Regression with Regularization</span>

## <span style="color:rebeccapurple">Notebook 3: Regularized model</span>

## <span style="color:rebeccapurple">ML workflow steps:</span>

1. State the problem
2. Gather the data
3. Split train-test sets
4. Pre-process the data
5. Establish a baseline
6. Choose a model
7. Train the model
8. Optimize the model
9. Validate the model
10. Predict unknown data points using the model
11. Interpret and evaluate the model

In [ ]:
# Import required packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn

In [ ]:
# setting some figure display paramaters
sns.set_context('notebook')
sns.set_style('white', {'axes.linewidth': 0.5})
matplotlib.rcParams.update(matplotlib.rcParamsDefault)

plt.rcParams['figure.dpi'] = 150
plt.rcParams['xtick.major.size'] = 3
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['legend.edgecolor'] = 'w'

## <span style="color:rebeccapurple">Why regularization?</span></h1>

We are going to use the same approach as notebook 1, but this time we will add **regularization**  to the model to see if it can be improved.

Regularization adds a penalty during model training, such that larger weights (coefficients) are penalized. Think of regularization as a method to reduce the dependence of the weights on the data, and to prevent weights from exploding in the presence of extreme outliers.

Remember the linear regressor minimizes the RSS. Now if you add a weight penalty the total "error" looks like:
$$ 
RSS + penalty  \\
= (y_{pred} - y_{obs})^2 + penalty  \\
= (\beta_0 + \sum_{i=1}^N \beta_i x_i - y_{obs})^2 + penalty \\
$$

Usually, we use either the L1 norm (lasso) or the L2 norm (ridge) (or a combination of the two) of the model weights as the penalty. 
$$
L1 = \lambda \sum_{i=1}^N\ \|\beta_i\|
$$

$$
L2 = \lambda \sum_{i=1}^N\beta_i^2
$$

Here, $\lambda$ is the tuning parameter that decides how much we want to penalize the flexibility of our model.

## <span style="color:rebeccapurple">Setup for linear model</span></h1>
Let's re-do the Steps 1-6 as before

<h4><span style="color:blue">Google Colab users only -- un-comment the code lines below and run them to download the dataset and read it</span></h4>

In [ ]:
# !wget https://raw.githubusercontent.com/nuitrcs/scikit-learn-workshop/main/data/penguins.csv
# df = pd.read_csv('penguins.csv')

In [ ]:
# 1 - task : predict body mass from penguin features
# 2 - load data
df = pd.read_csv('data/penguins.csv')

In [ ]:
# 3 - split features and target
X = df.drop(columns="body_mass_g")
y = df.body_mass_g

# 4 - split train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# 5 - pre-process the data - numeric
numeric_cols = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train[numeric_cols])

numeric_X_train = scaler.transform(X_train[numeric_cols])
numeric_X_test  = scaler.transform(X_test[numeric_cols])

In [ ]:
# 5 - pre-process the data - categorical
categorical_cols = ['species', 'island', 'sex', 'year']

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

ohe.fit(X_train[categorical_cols])

ohe_X_train = ohe.transform(X_train[categorical_cols]) # transform train set
ohe_X_test = ohe.transform(X_test[categorical_cols])   # transform test set

# convert to numpy array
ohe_X_train = ohe_X_train.toarray()
ohe_X_test = ohe_X_test.toarray()

In [ ]:
# 5 - stack the processed columns together
processed_X_train = np.hstack((numeric_X_train, ohe_X_train))
processed_X_test = np.hstack((numeric_X_test, ohe_X_test))

processed_X_train.shape, processed_X_test.shape

For Step 6 - we can now assume our old model (without regularization) is the "baseline" model

## <span style="color:rebeccapurple">7. Train the model</span></h1>

We will train 3 models, one with L1 (Lasso), one with L2 (ridge) and one with both L1 and L2 (elastic net) regularization

## <span style="color:rebeccapurple">8. Optimize the model</span></h1>

In [ ]:
from sklearn import linear_model

In [ ]:
model0 = linear_model.LinearRegression()

In [ ]:
alpha = 0.1

In [ ]:
model1 = linear_model.Lasso(alpha=alpha) # alpha controls regularization strength

In [ ]:
model2 = linear_model.Ridge(alpha=alpha)

In [ ]:
model3 = linear_model.ElasticNet(alpha=alpha, l1_ratio=0.5)

In [ ]:
# fit the models to the training data
model0.fit(processed_X_train, y_train)
model1.fit(processed_X_train, y_train)
model2.fit(processed_X_train, y_train)
model3.fit(processed_X_train, y_train)

In [ ]:
ohe.categories_

In [ ]:
# get the intercept and model coefficients
feature_names = numeric_cols + list(ohe.categories_[0]) + list(ohe.categories_[1]) + list(ohe.categories_[2]) + list(ohe.categories_[3])

coefs = pd.DataFrame({"Lasso" : model1.coef_, 
                      "Ridge" : model2.coef_, 
                      "Elastic_net" : model3.coef_,
                      "OLS": model0.coef_, })
coefs.index = feature_names

coefs

<span style="color:#DC537D"><font size="+1">What do you notice about the coefficients?</font></span>

In [ ]:
coefs["feature"] = feature_names

In [ ]:
# plot the model coefficients for all models
coefs.plot(x="feature", kind="barh", stacked=False, title="Model coefficients")

In [ ]:
# plot the model coefficients for L1 and L2 models only
coefs[["feature", "Lasso", "Ridge"]].plot(x="feature", kind="barh", stacked=False, title="Model coefficients")

<span style="color:#DC537D"><font size="+1">Do any values seem missing in the plot above?</font></span>

## <span style="color:rebeccapurple">10. Predict values for the test set  <br> 11. Evaluate performance</span></h1>

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

for model, name in zip([model0, model1,  model2, model3], ["OLS", "Lasso", "Ridge", "Elastic Net"]):
    print("\n")
    print(name)
    
    # predict
    y_predicted_test = model.predict(processed_X_test)
    
    # evaluate performance
    print("test R2 : ", round(r2_score(y_test, y_predicted_test), 3))
    print("test MSE : ", int(mean_squared_error(y_test, y_predicted_test)))

<span style="color:#DC537D"><font size="+2">How does alpha (lambda) affect the results?</font></span>
<br> Change the value of alpha across 0.1, 10, 100 for lasso and ridge regression models and plot the results. <br> Also calculate the model performance across these different values of alpha.

In [ ]:
coefficient_dataframe_list = []

for alpha in [0.1, 10, 100]:
    # define model hyperparams
    model1 = linear_model.Lasso(alpha=alpha)
    model2 = linear_model.Ridge(alpha=alpha)
    
    # fit the model on train data
    model1.fit(processed_X_train, y_train)
    model2.fit(processed_X_train, y_train)
    
    #get coefficients table
    coefs = pd.DataFrame({"Lasso" : model1.coef_, "Ridge" : model2.coef_, })
    coefs["feature"] = feature_names
    coefs["alpha"] = alpha
    
    # append to list
    coefficient_dataframe_list.append(coefs)
    
# concat the list of dataframes
final_coefs = pd.concat(coefficient_dataframe_list)

In [ ]:
final_coefs

In [ ]:
# plot the results for alpha = 0.1
plot_df = final_coefs.loc[final_coefs.alpha==0.1, ["Lasso", "Ridge", "feature"]]
plot_df.plot()

In [ ]:
# plot the results for alpha = 10
plot_df = final_coefs.loc[final_coefs.alpha==10, ["Lasso", "Ridge", "feature"]]
plot_df.plot(x="feature", kind="barh", stacked=False, title="Model coefficients")

In [ ]:
# plot the results for alpha=100
plot_df = final_coefs.loc[final_coefs.alpha==100, ["Lasso", "Ridge", "feature"]]
plot_df.plot(x="feature", kind="barh", stacked=False, title="Model coefficients")